In [1]:
from ase import Atoms
from ase.neighborlist import neighbor_list
from ase.neighborlist import NeighborList
import numpy as np
from ase.io import read, write
import math
from collections import defaultdict


In [2]:
def divmod_fortran(a, b):
    q = a // b  # Integer division in Python
    r = a % b   # Remainder (modulo operation)
    return q, r

In [3]:
def split_index(index, nx, ny, nz):
    tmp1, ix = divmod(index - 1, nx)
    tmp2, iy = divmod(tmp1, ny)
    iatom, iz = divmod(tmp2, nz)
    
    ix += 1
    iy += 1
    iz += 1
    iatom += 1

    # Adjust indices for periodic boundary conditions.
    # If an index is greater than the ceiling of (n/2), subtract n to wrap around
    if ix > math.ceil(nx / 2):
        ix = ix - nx
    if iy > math.ceil(ny / 2):
        iy = iy - ny
    if iz > math.ceil(nz / 2):
        iz = iz - nz

    return ix, iy, iz, iatom

In [4]:
supercell_filename = "infile_gan_supercell.poscar"
atoms = read(supercell_filename, format='vasp')
atoms

Atoms(symbols='Ga288N288', pbc=True, cell=[[19.141437506897706, 0.0, 0.0], [-9.570718753451857, 16.576971145925192, 0.0], [0.0, -0.0, 20.76077289420535]])

In [5]:
cutoff = 7.0
i, j, d = neighbor_list('ijd', atoms, cutoff)

all_neighbors = defaultdict(list)
all_distance = defaultdict(list)
for idx, neighbor_idx, distance  in zip(i, j, d):
    all_neighbors[idx].append(neighbor_idx)
    all_distance[idx].append(distance)

In [6]:
qx, qy, qz = 6, 6, 4

for atom_idx in range(len(all_neighbors[0])):
    center_atom, neighbor_atom = 0, all_neighbors[0][atom_idx]
    ix1, iy1, iz1, iatom1 = split_index(center_atom+1, qx, qy, qz)
    ix2, iy2, iz2, iatom2 = split_index(neighbor_atom+1, qx, qy, qz)
    lattice_disp = [ix2 - ix1, iy2 - iy1, iz2 - iz1]
    print(atom_idx+1, center_atom+1, neighbor_atom+1, all_distance[0][atom_idx], "|", ix1, iy1, iz1,  "|", ix2, iy2, iz2,  "|", lattice_disp)

1 1 312 4.914857510228287 | 1 1 1 | 0 -2 1 | [-1, -3, 0]
2 1 302 6.113045814932744 | 1 1 1 | 2 3 1 | [1, 2, 0]
3 1 303 4.542632215731719 | 1 1 1 | 3 3 1 | [2, 2, 0]
4 1 8 3.1823056567923103 | 1 1 1 | 2 2 1 | [1, 1, 0]
5 1 304 1.9495043300913046 | 1 1 1 | -2 3 1 | [-3, 2, 0]
6 1 7 5.190193223551336 | 1 1 1 | 1 2 1 | [0, 1, 0]
7 1 16 3.182305656791755 | 1 1 1 | -2 3 1 | [-3, 2, 0]
8 1 296 1.949504330092211 | 1 1 1 | 2 2 1 | [1, 1, 0]
9 1 351 4.542632215731367 | 1 1 1 | 3 -1 2 | [2, -2, 1]
10 1 343 4.542632215731593 | 1 1 1 | 1 -2 2 | [0, -3, 1]
11 1 344 4.914857510229093 | 1 1 1 | 2 -2 2 | [1, -3, 1]
12 1 350 6.895430207163918 | 1 1 1 | 2 -1 2 | [1, -2, 1]
13 1 72 5.521080193770799 | 1 1 1 | 0 0 2 | [-1, -1, 1]
14 1 64 4.50607344587843 | 1 1 1 | -2 -1 2 | [-3, -2, 1]
15 1 408 6.671662578296718 | 1 1 1 | 0 2 0 | [-1, 1, -1]
16 1 360 4.914857510227956 | 1 1 1 | 0 0 2 | [-1, -1, 1]
17 1 56 5.5210801937718115 | 1 1 1 | 2 -2 2 | [1, -3, 1]
18 1 295 3.233864381974076 | 1 1 1 | 1 2 1 | [0, 1, 0

In [7]:
diff = atoms.get_positions()[1-1] - atoms.get_positions()[2-1]
diff = diff - np.dot(np.round(np.linalg.solve(atoms.get_cell().T, diff)), atoms.get_cell())
distance = np.linalg.norm(diff)
distance

3.1823056567923076

In [ ]:
qx, qy, qz = 6, 6, 4
cutoff = 7.0
ss_phonopy = 576
center_atom, within_cutff = 1, 0

for j in range(1, ss_phonopy + 1):
    ix1, iy1, iz1, iatom1 = split_index(center_atom, qx, qy, qz)
    ix2, iy2, iz2, iatom2 = split_index(j, qx, qy, qz)
    diff = atoms.get_positions()[center_atom-1] - atoms.get_positions()[j-1]
    diff = diff - np.dot(np.round(np.linalg.solve(atoms.get_cell().T, diff)), atoms.get_cell())
    distance = np.linalg.norm(diff)
    if distance < cutoff:
        within_cutff+=1
        lattice_disp = [ix2 - ix1, iy2 - iy1, iz2 - iz1]
        print(center_atom, j, distance, "|", ix1, iy1, iz1,  "|", ix2, iy2, iz2,  "|", lattice_disp, "|", iatom1, iatom2)

1 1 0.0 | 1 1 1 | 1 1 1 | [0, 0, 0] | 1 1
1 2 3.1823056567923076 | 1 1 1 | 0 1 1 | [-1, 0, 0] | 1 1
1 3 5.190193223551337 | 1 1 1 | 1 0 1 | [0, -1, 0] | 1 1
1 7 5.190193223551336 | 1 1 1 | 1 0 0 | [0, -1, -1] | 1 1
1 8 3.1823056567923103 | 1 1 1 | 0 0 0 | [-1, -1, -1] | 1 1
1 9 3.1902395844831295 | 1 1 1 | 1 1 1 | [0, 0, 0] | 1 2
1 10 3.182305656791752 | 1 1 1 | 0 1 1 | [-1, 0, 0] | 1 2
1 11 6.092268403821462 | 1 1 1 | 1 0 1 | [0, -1, 0] | 1 2
1 15 6.092268403821462 | 1 1 1 | 1 0 0 | [0, -1, -1] | 1 2
1 16 3.182305656791755 | 1 1 1 | 0 0 0 | [-1, -1, -1] | 1 2
1 17 6.380479168966261 | 1 1 1 | 1 1 1 | [0, 0, 0] | 1 3
1 18 5.521080193771091 | 1 1 1 | 0 1 1 | [-1, 0, 0] | 1 3
1 24 5.521080193771093 | 1 1 1 | 0 0 0 | [-1, -1, -1] | 1 3
1 33 6.380479168966262 | 1 1 1 | 1 1 1 | [0, 0, 0] | 1 5
1 41 3.1902395844831317 | 1 1 1 | 1 1 1 | [0, 0, 0] | 1 6
1 42 5.521080193772054 | 1 1 1 | 0 1 1 | [-1, 0, 0] | 1 6
1 43 6.092268403821464 | 1 1 1 | 1 0 1 | [0, -1, 0] | 1 6
1 47 6.092268403821463 | 1 

In [9]:
within_cutff

125